In [60]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [61]:
### Load the trained model, scaler pickle,onehod
model = load_model('/Users/phupwinteaindray/Documents/ANNClassification/model.h5')

#Load the encoder and scaler
with open('/Users/phupwinteaindray/Documents/ANNClassification/onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo = pickle.load(file)

with open("/Users/phupwinteaindray/Documents/ANNClassification/lable_encoder_gender.pkl",'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('/Users/phupwinteaindray/Documents/ANNClassification/scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [62]:
#Example input data
input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [63]:
input_data = pd.DataFrame([input_data])

In [64]:
#One-hot encdoe Geography
geo_encoded = label_encoder_geo.transform([input_data['Geography']])

geo_encoded_df = pd.DataFrame(
    geo_encoded,
    columns=label_encoder_geo.get_feature_names_out(['Geography'])
)


/Users/phupwinteaindray/Documents/ANNClassification/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [65]:
input_df = pd.concat([input_data.reset_index(drop=True),geo_encoded_df],axis=1)
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [66]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

In [67]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


### Concatination one hot encoded data

In [72]:
geo_cols = [col for col in input_df.columns if col.startswith('Geography_')]

input_df = input_df.drop(columns=geo_cols)


In [75]:
input_df = pd.concat([input_df, geo_encoded_df], axis=1)

input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


### Scaling the input data

In [69]:
scaler.feature_names_in_


array(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [76]:
input_scaled = scaler.transform(input_df)

In [77]:
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [78]:
#Predict churn

prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 24ms/step


array([[0.03355621]], dtype=float32)

In [79]:
prediction_proba = prediction[0][0]
prediction_proba

0.033556208

In [80]:
if prediction_proba>0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
